In [36]:
from scipy.io import loadmat
import numpy as np
import scipy.optimize as opt
import pandas as pd

In [37]:
data = loadmat('ex4data1.mat')
X = data['X']
y = data['y']

In [38]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [39]:
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10

In [40]:
weights = loadmat('ex4weights.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']

In [41]:
lmbda = 1

In [42]:
m = len(y)
ones = np.ones((m,1))
a1 = np.hstack((ones, X))
a2 = sigmoid(a1 @ theta1.T)
a2 = np.hstack((ones, a2))
h = sigmoid(a2 @ theta2.T)

In [43]:
J = np.zeros((num_labels, 1))

In [44]:
y_d = pd.get_dummies(y.flatten())

In [45]:
def nnCostFunction(h, y_d):
    temp1 = np.multiply(y_d, np.log(h))
    temp2 = np.multiply(1-y_d, np.log(1-h))
    temp3 = np.sum(temp1 + temp2)
    return np.sum(temp3 / (-m))

In [46]:
J = nnCostFunction(h, y_d)
J

0.28762916516131876

In [47]:
def nnCostFunctionReg(theta1, theta2):
    sum1 = np.sum(np.sum(np.power(theta1[:,1:],2), axis = 1))
    sum2 = np.sum(np.sum(np.power(theta2[:,1:],2), axis = 1))
    return (sum1 + sum2) * lmbda / (2*m)

In [48]:
lr = nnCostFunctionReg(theta1, theta2)
lr

0.09614069392960475

In [49]:
J + lr

0.38376985909092354

In [50]:
def sigmoidGrad(z):
    return np.multiply(sigmoid(z), 1-sigmoid(z))

In [51]:
def randInitializeWeights(L_in, L_out):
    epsilon = 0.12
    return np.random.rand(L_out, L_in+1) * 2 * epsilon - epsilon

In [52]:
initial_theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_theta2 = randInitializeWeights(hidden_layer_size, num_labels)

In [53]:
delta1 = np.zeros(initial_theta1.shape)
delta2 = np.zeros(initial_theta2.shape)

In [54]:
for i in range(X.shape[0]):
    ones = np.ones(1)
    a1 = np.hstack((ones, X[i]))
    z2 = a1 @ initial_theta1.T
    a2 = np.hstack((ones, sigmoid(z2)))
    z3 = a2 @ initial_theta2.T
    a3 = sigmoid(z3)
    
    d3 = a3 - y_d.iloc[i,:][np.newaxis,:]
    
    z2 = np.hstack((ones, z2))
    
    d2 = np.multiply(initial_theta2.T @ d3.T, sigmoidGrad(z2).T[:,np.newaxis])
    delta1 = delta1 + d2[1:,:] @ a1[np.newaxis,:]
    delta2 = delta2 + d3.T @ a2[np.newaxis,:]

In [55]:
delta1 /= m
delta2 /= m

In [56]:
nn_params = np.hstack((theta1.ravel(order='F'), theta2.ravel(order='F')))

In [57]:
def nnCostFunc(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda):
    theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (hidden_layer_size, input_layer_size+1), 'F')
    theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (num_labels, hidden_layer_size+1), 'F')

    m = len(y)
    ones = np.ones((m,1))
    a1 = np.hstack((ones, X))
    a2 = sigmoid(a1 @ theta1.T)
    a2 = np.hstack((ones, a2))
    h = sigmoid(a2 @ theta2.T)
    
    y_d = pd.get_dummies(y.flatten())
    
    temp1 = np.multiply(y_d, np.log(h))
    temp2 = np.multiply(1-y_d, np.log(1-h))
    temp3 = np.sum(temp1 + temp2)
    
    sum1 = np.sum(np.sum(np.power(theta1[:,1:],2), axis = 1))
    sum2 = np.sum(np.sum(np.power(theta2[:,1:],2), axis = 1))
    
    return np.sum(temp3 / (-m)) + (sum1 + sum2) * lmbda / (2*m)

In [58]:
nnCostFunc(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda)

0.38376985909092354

In [59]:
initial_theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_theta2 = randInitializeWeights(hidden_layer_size, num_labels)
nn_initial_params = np.hstack((initial_theta1.ravel(order='F'), initial_theta2.ravel(order='F')))

In [60]:
nnCostFunc(nn_initial_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda)

6.897800257249203

In [61]:
def nnGrad(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda):
    
    initial_theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (hidden_layer_size, input_layer_size+1), 'F')
    initial_theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (num_labels, hidden_layer_size+1), 'F')
    y_d = pd.get_dummies(y.flatten())
    delta1 = np.zeros(initial_theta1.shape)
    delta2 = np.zeros(initial_theta2.shape)
    m = len(y)
    
    for i in range(X.shape[0]):
        ones = np.ones(1)
        a1 = np.hstack((ones, X[i]))
        z2 = a1 @ initial_theta1.T
        a2 = np.hstack((ones, sigmoid(z2)))
        z3 = a2 @ initial_theta2.T
        a3 = sigmoid(z3)

        d3 = a3 - y_d.iloc[i,:][np.newaxis,:]
        z2 = np.hstack((ones, z2))
        d2 = np.multiply(initial_theta2.T @ d3.T, sigmoidGrad(z2).T[:,np.newaxis])
        delta1 = delta1 + d2[1:,:] @ a1[np.newaxis,:]
        delta2 = delta2 + d3.T @ a2[np.newaxis,:]
        
    delta1 /= m
    delta2 /= m
    #print(delta1.shape, delta2.shape)
    delta1[:,1:] = delta1[:,1:] + initial_theta1[:,1:] * lmbda / m
    delta2[:,1:] = delta2[:,1:] + initial_theta2[:,1:] * lmbda / m
        
    return np.hstack((delta1.ravel(order='F'), delta2.ravel(order='F')))

In [62]:
nn_backprop_Params = nnGrad(nn_initial_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda)

In [63]:
theta_opt = opt.fmin_cg(maxiter = 50, f = nnCostFunc, x0 = nn_initial_params, fprime = nnGrad, args = (input_layer_size, hidden_layer_size, num_labels, X, y.flatten(), lmbda))

         Current function value: 0.439121
         Iterations: 50
         Function evaluations: 108
         Gradient evaluations: 108


In [64]:
theta1_opt = np.reshape(theta_opt[:hidden_layer_size*(input_layer_size+1)], (hidden_layer_size, input_layer_size+1), 'F')
theta2_opt = np.reshape(theta_opt[hidden_layer_size*(input_layer_size+1):], (num_labels, hidden_layer_size+1), 'F')

In [65]:
nnCostFunc(theta_opt, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda)

0.43912115362687193

In [66]:
def checkGradient(nn_initial_params,nn_backprop_Params,input_layer_size, hidden_layer_size, num_labels,myX,myy,mylambda=0.):
    myeps = 0.0001
    flattened = nn_initial_params
    flattenedDs = nn_backprop_Params
    n_elems = len(flattened) 
    #Pick ten random elements, compute numerical gradient, compare to respective D's
    for i in range(10):
        x = int(np.random.rand()*n_elems)
        epsvec = np.zeros((n_elems,1))
        epsvec[x] = myeps

        cost_high = nnCostFunc(flattened + epsvec.flatten(),input_layer_size, hidden_layer_size, num_labels,myX,myy,mylambda)
        cost_low  = nnCostFunc(flattened - epsvec.flatten(),input_layer_size, hidden_layer_size, num_labels,myX,myy,mylambda)
        mygrad = (cost_high - cost_low) / float(2*myeps)
        print("Element: {0}. Numerical Gradient = {1:.9f}. BackProp Gradient = {2:.9f}.".format(x,mygrad,flattenedDs[x]))

In [67]:
checkGradient(nn_initial_params,nn_backprop_Params,input_layer_size, hidden_layer_size, num_labels,X,y,lmbda)

Element: 3225. Numerical Gradient = 0.005045094. BackProp Gradient = 0.005045094.
Element: 3746. Numerical Gradient = -0.003467335. BackProp Gradient = -0.003467335.
Element: 1469. Numerical Gradient = -0.000024397. BackProp Gradient = -0.000024397.
Element: 493. Numerical Gradient = 0.000017703. BackProp Gradient = 0.000017703.
Element: 2560. Numerical Gradient = 0.000026983. BackProp Gradient = 0.000026983.
Element: 2839. Numerical Gradient = 0.001789760. BackProp Gradient = 0.001789760.
Element: 1620. Numerical Gradient = 0.000003114. BackProp Gradient = 0.000003114.
Element: 8746. Numerical Gradient = -0.000125304. BackProp Gradient = -0.000125304.
Element: 6705. Numerical Gradient = 0.004496578. BackProp Gradient = 0.004496578.
Element: 5604. Numerical Gradient = -0.002057464. BackProp Gradient = -0.002057464.


In [68]:
def predict(theta1, theta2, X, y):
    m = len(y)
    ones = np.ones((m,1))
    a1 = np.hstack((ones, X))
    a2 = sigmoid(a1 @ theta1.T)
    a2 = np.hstack((ones, a2))
    h = sigmoid(a2 @ theta2.T)
    return np.argmax(h, axis = 1) + 1

In [69]:
pred = predict(theta1_opt, theta2_opt, X, y)

In [70]:
np.mean(pred == y.flatten()) * 100

96.5